<a href="https://colab.research.google.com/github/Vakhranev/Gramota/blob/main/%D0%A1%D0%BF%D1%80%D0%B0%D0%B2%D0%BE%D1%87%D0%BD%D0%B8%D0%BA%D0%B8%20%D0%B2%20HTML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transliterate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 2.7 MB/s eta 0:00:00


In [37]:
import os
import re
import xml.etree.ElementTree as ET
from transliterate import translit

def parse_chapters_subchapters_paragraphs(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    structure = {}
    current_chapter = None
    current_subchapter = None
    current_paragraph = None

    for para in root.findall('paragraph'):
        text = para.text.strip()

        # Ищем заголовки глав
        if text == "1.  Орфография" or text == "2. Пунктуация":
            current_chapter = text
            structure[current_chapter] = {}
            current_subchapter = None
            current_paragraph = None

        # Ищем заголовки подглав (X.Y.)
        elif re.match(r'^\d+\.\d+\.\s', text):
            current_subchapter = text
            if current_chapter:  # Убеждаемся, что есть глава
                structure[current_chapter][current_subchapter] = {'text': []}
                current_paragraph = None

        # Ищем заголовки параграфов (X.Y.Z. Текст)
        elif re.match(r'^\d+\.\d+\.\d+\.\s', text):
            current_paragraph = text
            if current_chapter and current_subchapter:  # Проверяем существование главы и подглавы
                if current_paragraph not in structure[current_chapter][current_subchapter]:
                    structure[current_chapter][current_subchapter][current_paragraph] = []

        else:
            if current_chapter and current_subchapter:  # Проверяем, что глава и подглава существуют
                if current_paragraph:
                    # Создаём список, если он не существует
                    if current_paragraph not in structure[current_chapter][current_subchapter]:
                        structure[current_chapter][current_subchapter][current_paragraph] = []
                    structure[current_chapter][current_subchapter][current_paragraph].append(text)
                else:
                    structure[current_chapter][current_subchapter]['text'].append(text)

    return structure

def sanitize_filename(name):
    return translit(re.sub(r'[^a-zA-Z0-9_]', '_', name.lower()), 'ru', reversed=True)

def generate_html(structure, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for chapter, subchapters in structure.items():
        chapter_dir = os.path.join(output_dir, sanitize_filename(chapter))
        os.makedirs(chapter_dir, exist_ok=True)

        for subchapter, sections in subchapters.items():
            subchapter_dir = os.path.join(chapter_dir, sanitize_filename(subchapter))
            os.makedirs(subchapter_dir, exist_ok=True)

            # Записываем текст подглавы, если он есть
            if 'text' in sections and sections['text']:
                with open(os.path.join(subchapter_dir, "text.html"), 'w', encoding='utf-8') as f:
                    f.write(f'<html><head><title>{subchapter}</title></head><body>')
                    f.write(f'<h1>{chapter}</h1>')
                    f.write(f'<h2>{subchapter}</h2>')
                    f.write(f'<p>{"<br>".join(sections["text"])}</p>')
                    f.write('</body></html>')

            for section, paragraphs in sections.items():
                if section == 'text':
                    continue  # Текст уже обработан выше

                section_file = os.path.join(subchapter_dir, sanitize_filename(section) + '.html')
                with open(section_file, 'w', encoding='utf-8') as f:
                    f.write(f'<html><head><title>{section}</title></head><body>')
                    f.write(f'<h1>{chapter}</h1>')
                    f.write(f'<h2>{subchapter}</h2>')
                    f.write(f'<h3>{section}</h3>')
                    for p in paragraphs:
                        f.write(f'<p>{p}</p>')
                    f.write('</body></html>')

    print("HTML-файлы созданы в", output_dir)

xml_file = "Valgina_Orfografiya_i_punktuatsiya.xml"
output_dir = "output_html"

structure = parse_chapters_subchapters_paragraphs(xml_file)
print("Структура:", structure)
generate_html(structure, output_dir)

Структура: {'1.  Орфография': {'1.1. Правописание гласных в корне слова': {'text': [], '1.1.1. Проверяемые безударные гласные': ['В безударном положении в корне пишется та же гласная, что и в соответствующем', 'ударном слоге однокорневого слова: закалить (закалка)  – заколоть (заколка), запевать \n(запеть)  – запивать (запить), задрожать (дрожь)  – раздражать (дразнит), иссякать \n(иссякнуть)  – иссекать (иссечь), озимь (зимний)  – оземь (о землю), приведение \n(приведший)  – привидение (видеть), примерять (примерить)  – примирять (мир), \nразвеваться (веять)  – развиваться (развитие), старожил (старый житель)  – сторожил \n(сторож).', 'Примечание 1. Если в корне пишется ё, то в безударном положении в однокорневых \nсловах следует писать е: испещрённый (пёстрый), звездопад (звёзды), весенний (вёсны), \nгнездится (гнёзда) [О правописании е(ё)-о в корне после шипящих и после ц см. раздел \n1.1.4.].', 'Примечание 2. Нельзя проверять безударные а – о в корнях глаголов совершенного \nвида ф

In [38]:
import shutil

# Указываем имя папки, которую нужно заархивировать
folder_to_zip = "output_html"
zip_filename = "output_html.zip"

# Создаём ZIP-архив
shutil.make_archive(zip_filename.replace(".zip", ""), 'zip', folder_to_zip)

print(f"Архив {zip_filename} создан.")

Архив output_html.zip создан.
